OS Lab Assignment 1: Process Creation and Management

In [1]:
# Import all libraries
import os
import sys
import time
import subprocess

In [2]:
#Task 1: Process Creation Utility (os.fork(), os.wait())

def task1(n=3):
    print(f"Task 1: Creating {n} Child Processes")
    print(f"Parent process (PID: {os.getpid()}) is starting.")
    
    child_pids = []
    for i in range(n):
        pid = os.fork()
        
        if pid > 0:
            child_pids.append(pid)
            
        elif pid == 0:
            my_pid = os.getpid()
            my_ppid = os.getppid()
            print(f"[Child {i+1}] PID: {my_pid}, Parent PID: {my_ppid}, Message: Hello from Child {i+1}")
            
            os._exit(0) 
            
        else:
            print("Fork failed!", file=sys.stderr)
            sys.exit(1)
    
    print(f"\nParent (PID: {os.getpid()}) is waiting for {n} children")
    
    for _ in range(n):
        try:
            (reaped_pid, status) = os.wait()
            print(f"Parent: Reaped child with PID {reaped_pid}. Status: {status}")
        except ChildProcessError:
            pass
            
    print("Task 1: All children terminated. Parent exiting.")

In [12]:
task1(n=3)

Task 1: Creating 3 Child Processes
Parent process (PID: 58636) is starting.
[Child 1] PID: 58756, Parent PID: 58636, Message: Hello from Child 1Task 1: Creating 3 Child Processes
Parent process (PID: 58636) is starting.
[Child 2] PID: 58757, Parent PID: 58636, Message: Hello from Child 2Task 1: Creating 3 Child Processes
Parent process (PID: 58636) is starting.
[Child 3] PID: 58758, Parent PID: 58636, Message: Hello from Child 3


Task 1: Creating 3 Child Processes
Parent process (PID: 58636) is starting.

Parent (PID: 58636) is waiting for 3 children
Parent: Reaped child with PID 58756. Status: 0
Parent: Reaped child with PID 58758. Status: 0
Parent: Reaped child with PID 58757. Status: 0
Task 1: All children terminated. Parent exiting.


/var/folders/35/t2bsrjf13f153z8y1s_955fw0000gn/T/ipykernel_58636/2005507709.py:9: DeprecationWarning: This process (pid=58636) is multi-threaded, use of fork() may lead to deadlocks in the child.
  pid = os.fork()


In [3]:
def task2(command="ls"):
    
    print(f"Task 2: Executing Command '{command}'")
    
    print("\nMethod 1: Using subprocess.run()")
    
    pid = os.fork()
    if pid == 0:
        print(f"[Child PID: {os.getpid()}] Executing '{command} -l':")
        try:
            subprocess.run([command, "-l"])
        except FileNotFoundError:
            print(f"Error: Command '{command}' not found.")
        os._exit(0) 
    else:
        os.wait() 
        print("Parent: Child finished subprocess.run().")

    print("\nMethod 2: Using os.execvp()")
    pid = os.fork()
    if pid == 0:
        print(f"[Child PID: {os.getpid()}] Using os.execvp() to become 'date'...")
        try:
            os.execvp("date", ["date"]) 
        except FileNotFoundError:
            print("Error: Command 'date' not found.")
            os._exit(1) 
    else:
        
        os.wait() 
        print("Parent: Child (which became 'date') finished.")
        
    print("Task 2: Finished.")


In [13]:
task2(command="ls")

Task 2: Executing Command 'ls'

Method 1: Using subprocess.run()
[Child PID: 58759] Executing 'ls -l':
Task 2: Executing Command 'ls'

Method 1: Using subprocess.run()
total 24
-rw-r--r--@ 1 jayantparashar  staff    0 Nov  1 20:50 Labs-assignment-1.ipynb
-rw-r--r--@ 1 jayantparashar  staff  652 Sep 23 11:38 fork.cpp
-rw-r--r--@ 1 jayantparashar  staff  820 Sep 23 11:44 fork2.cpp
drwxr-xr-x@ 8 jayantparashar  staff  256 Sep 23 11:55 output
-rw-r--r--@ 1 jayantparashar  staff  932 Sep 23 11:52 zombie.cpp
Parent: Child finished subprocess.run().

Method 2: Using os.execvp()
[Child PID: 58761] Using os.execvp() to become 'date'...
Task 2: Executing Command 'ls'

Method 1: Using subprocess.run()
total 24
-rw-r--r--@ 1 jayantparashar  staff    0 Nov  1 20:50 Labs-assignment-1.ipynb
-rw-r--r--@ 1 jayantparashar  staff  652 Sep 23 11:38 fork.cpp
-rw-r--r--@ 1 jayantparashar  staff  820 Sep 23 11:44 fork2.cpp
drwxr-xr-x@ 8 jayantparashar  staff  256 Sep 23 11:55 output
-rw-r--r--@ 1 jayantparas

/var/folders/35/t2bsrjf13f153z8y1s_955fw0000gn/T/ipykernel_58636/2023604381.py:7: DeprecationWarning: This process (pid=58636) is multi-threaded, use of fork() may lead to deadlocks in the child.
  pid = os.fork()
/var/folders/35/t2bsrjf13f153z8y1s_955fw0000gn/T/ipykernel_58636/2023604381.py:20: DeprecationWarning: This process (pid=58636) is multi-threaded, use of fork() may lead to deadlocks in the child.
  pid = os.fork()


In [6]:
def task3_zombie_process():
    
    print("Task 3: Simulating a ZOMBIE Process")
    
    pid = os.fork()
    
    if pid > 0:
        print(f"Parent (PID: {os.getpid()}) is sleeping for 30 seconds.")
        print(f"The child (PID: {pid}) will exit, but parent will NOT wait().")
        print("Run 'ps -el | grep defunct' or 'ps -el | grep Z' in another terminal to see the zombie.")
        time.sleep(30) 
        
        (reaped_pid, status) = os.wait()
        print(f"Parent finally reaping zombie child (PID: {reaped_pid}).")
        
    else:
        print(f"Child (PID: {os.getpid()}) is exiting immediately.")
        os._exit(0) 

def task3_orphan_process():
    
    print("Task 3: Simulating an ORPHAN Process")
    
    pid = os.fork()
    
    if pid > 0:
        print(f"Parent (PID: {os.getpid()}) is exiting in 2 seconds.")
        time.sleep(2)
        print("Parent is exiting now.")
        sys.exit(0) 
        
    else:
        my_pid = os.getpid()
        original_ppid = os.getppid()
        print(f"Child (PID: {my_pid}) started with Parent (PID: {original_ppid}).")
        
        print("Child is sleeping for 10 seconds.")
        time.sleep(10)
        
        new_ppid = os.getppid()
        print(f"Child (PID: {my_pid}) is awake. Original PPID: {original_ppid}, New PPID: {new_ppid}.")
        print("Child is exiting.")
        os._exit(0)

In [14]:
task3_zombie_process()

Task 3: Simulating a ZOMBIE Process
Child (PID: 58779) is exiting immediately.
Task 3: Simulating a ZOMBIE Process
Parent (PID: 58636) is sleeping for 30 seconds.
The child (PID: 58779) will exit, but parent will NOT wait().
Run 'ps -el | grep defunct' or 'ps -el | grep Z' in another terminal to see the zombie.


/var/folders/35/t2bsrjf13f153z8y1s_955fw0000gn/T/ipykernel_58636/1276709462.py:5: DeprecationWarning: This process (pid=58636) is multi-threaded, use of fork() may lead to deadlocks in the child.
  pid = os.fork()


Parent finally reaping zombie child (PID: 58779).


In [ ]:
task3_orphan_process()

Task 3: Simulating an ORPHAN Process
Child (PID: 58793) started with Parent (PID: 58636).
Child is sleeping for 10 seconds.
Task 3: Simulating an ORPHAN Process
Parent (PID: 58636) is exiting in 2 seconds.


/var/folders/35/t2bsrjf13f153z8y1s_955fw0000gn/T/ipykernel_58636/1276709462.py:24: DeprecationWarning: This process (pid=58636) is multi-threaded, use of fork() may lead to deadlocks in the child.
  pid = os.fork()


Parent is exiting now.


SystemExit: 0

Child (PID: 58793) is awake. Original PPID: 58636, New PPID: 58636.
Child is exiting.


In [10]:
def task4(target_pid):
    
    print(f"Task 4: Inspecting /proc/{target_pid}")
    
    try:
        print(f"\n[+] From /proc/{target_pid}/status:")
        with open(f"/proc/{target_pid}/status") as f:
            for line in f:
                if line.startswith("Name:") or line.startswith("State:") or line.startswith("VmSize:"):
                    print(f"  {line.strip()}")
                    
        print(f"\n[+] Executable Path from /proc/{target_pid}/exe:")
        exe_path = os.readlink(f"/proc/{target_pid}/exe")
        print(f"  {exe_path}")
        
        print(f"\n[+] Open File Descriptors from /proc/{target_pid}/fd:")
        fd_list = os.listdir(f"/proc/{target_pid}/fd")
        print(f"  Total: {len(fd_list)} open FDs.")
        print(f"  {fd_list}")
        
    except FileNotFoundError:
        print(f"Error: Process with PID {target_pid} not found (or /proc not available).", file=sys.stderr)
    except PermissionError:
        print(f"Error: Permission denied. Try running as root or on your own process.", file=sys.stderr)
        
    print("Task 4: Finished.")

In [17]:
my_pid = os.getpid()
print(f"No PID provided, running on self (PID: {my_pid})...")
task4(my_pid)

No PID provided, running on self (PID: 58636)...
Task 4: Inspecting /proc/58636

[+] From /proc/58636/status:
Task 4: Finished.


Error: Process with PID 58636 not found (or /proc not available).


In [11]:
def task5():
    
    print("Task 5: Process Prioritization (Nice)")

    print("Creating two children. Child 1 gets low priority (nice=10). Child 2 gets default (nice=0).")
    
    def cpu_intensive_task(label, nice_val):
        my_pid = os.getpid()
        
        os.nice(nice_val) 
        
        print(f"[Child {label}, PID: {my_pid}, Nice: {os.nice(0)}] Starting CPU task.")
        
        count = 0
        for _ in range(150_000_000): 
            count += 1
            
        print(f"[Child {label}, PID: {my_pid}] Finished.")
        os._exit(0)

    pid1 = os.fork()
    if pid1 == 0:
        cpu_intensive_task("1 (Low Priority)", 10) 

    pid2 = os.fork()
    if pid2 == 0:
        cpu_intensive_task("2 (Default Priority)", 0) 
    
    print("Parent waiting for both children...")
    os.waitpid(pid1, 0)
    print("Parent: Child 1 finished.")
    os.waitpid(pid2, 0)
    print("Parent: Child 2 finished.")
    print("Task 5: Finished.")